In [37]:
import argparse
import json
import tensorflow as tf
import numpy as np
import h5py
import copy
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
from evtk import hl
#from mayavi import mlab
#mlab.init_notebook('itk')

from fr3D.train.utils import setup_datasets
from fr3D.data.utils import get_normalization
from fr3D.models import ConvAutoencoder, ConvAutoencoderCGAN, ConvAutoencoderC

def mape_with_threshold(yp, yt, pcterror_threshold=np.inf, max_magnitude_threshold=0.0, eps=1e-7):
    pct_errors = 100*tf.abs((yp-yt)/(eps + yt))
    pcterror_mask = pct_errors < pcterror_threshold
    max_magnitude_mask = tf.logical_not(tf.abs(yt) < (max_magnitude_threshold*tf.reduce_max(tf.abs(yt))))
    filtering_indices = tf.where(tf.logical_and(pcterror_mask, max_magnitude_mask))
    filtered_pcterrors = tf.gather_nd(pct_errors, filtering_indices)
    return float(tf.reduce_mean(filtered_pcterrors))

def get_normalization_type(node_configs):
    normalization_spec = {'method': None}
    for node in node_configs:
        if node['nodetype'] == 'normalize':
            normalization_spec = node['normalization_spec']
            break
    normalizer = get_normalization(**normalization_spec, batch_mode=True)
    return normalizer

In [2]:
Re = 500
expt_variables = ['Pressure', 'U', 'V', 'W']

dataset_path = f'/fr3D/postprocessed/annulus_64.h5'

#ConvAutoencoderC
experiment_configs = {expt_variable:f'/fr3D/configs/training/ConvAutoencoderC_{expt_variable}.json' for expt_variable in expt_variables}
weights_paths = {expt_variable:f'/storage/weights{Re}/ConvAutoencoderC_{expt_variable}_Annulus64/ConvAutoencoderC_{expt_variable}_Annulus64.h5' for expt_variable in expt_variables}



datasetf = h5py.File(dataset_path,'r')

shuf_buf = 1

train_datasets = {}
test_datasets = {}
sensor_shapes = {}
full_field_shapes = {}
normalizers = {}

for expt_variable in experiment_configs:
    config = json.load(open(experiment_configs[expt_variable],'r'))
    train_datasets[expt_variable], test_datasets[expt_variable] = setup_datasets(config, dataset_path, shuf_buf, case_names=True, evaluation=True)
    sensor_shapes[expt_variable] = train_datasets[expt_variable].element_spec[0][0].shape
    full_field_shapes[expt_variable] = train_datasets[expt_variable].element_spec[0][1].shape
    normalizers[expt_variable] = get_normalization_type(config['dataset']['node_configurations'])
    
dataset_iterators = {v: iter(test_datasets[v]) for v in test_datasets}

Training geometries: ('shape_69', 'shape_33', 'shape_37', 'shape_374', 'shape_239', 'shape_224', 'shape_160', 'shape_237', 'shape_135', 'cylinder', 'shape_152', 'shape_186', 'shape_92', 'shape_21', 'shape_58', 'shape_110', 'shape_88', 'shape_59', 'shape_14', 'shape_197', 'shape_55', 'shape_8', 'shape_170', 'shape_234', 'shape_43', 'shape_149', 'shape_228', 'shape_48', 'shape_130', 'shape_90', 'shape_137', 'shape_84', 'shape_361', 'shape_182', 'shape_68', 'shape_113', 'shape_244', 'shape_327', 'shape_216', 'shape_15', 'shape_63', 'shape_213', 'shape_126', 'shape_236', 'shape_50', 'shape_18', 'shape_157', 'shape_89', 'shape_17', 'shape_355', 'shape_29', 'shape_146', 'shape_164', 'shape_11', 'shape_150', 'shape_220', 'shape_13', 'shape_60', 'shape_12', 'shape_369', 'shape_218', 'shape_78', 'shape_331', 'shape_97', 'shape_329', 'shape_31', 'shape_39', 'shape_241', 'shape_41', 'shape_346', 'shape_94', 'shape_61', 'shape_23', 'shape_339', 'shape_255', 'shape_83', 'shape_338', 'shape_54', 'sh

2022-12-16 14:14:56.882266: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-12-16 14:14:56.882435: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2022-12-16 14:14:57.196146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 14:14:58.083142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38278 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0
2022-12-16 14:14:58.084654: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] 

Training geometries: ('shape_69', 'shape_33', 'shape_37', 'shape_374', 'shape_239', 'shape_224', 'shape_160', 'shape_237', 'shape_135', 'cylinder', 'shape_152', 'shape_186', 'shape_92', 'shape_21', 'shape_58', 'shape_110', 'shape_88', 'shape_59', 'shape_14', 'shape_197', 'shape_55', 'shape_8', 'shape_170', 'shape_234', 'shape_43', 'shape_149', 'shape_228', 'shape_48', 'shape_130', 'shape_90', 'shape_137', 'shape_84', 'shape_361', 'shape_182', 'shape_68', 'shape_113', 'shape_244', 'shape_327', 'shape_216', 'shape_15', 'shape_63', 'shape_213', 'shape_126', 'shape_236', 'shape_50', 'shape_18', 'shape_157', 'shape_89', 'shape_17', 'shape_355', 'shape_29', 'shape_146', 'shape_164', 'shape_11', 'shape_150', 'shape_220', 'shape_13', 'shape_60', 'shape_12', 'shape_369', 'shape_218', 'shape_78', 'shape_331', 'shape_97', 'shape_329', 'shape_31', 'shape_39', 'shape_241', 'shape_41', 'shape_346', 'shape_94', 'shape_61', 'shape_23', 'shape_339', 'shape_255', 'shape_83', 'shape_338', 'shape_54', 'sh

In [167]:
def circular_pad(x, coords=True):
    if coords:
        r = np.zeros((x.shape[0]+2, x.shape[1]+2, x.shape[2]))
    else:
        r = np.zeros((x.shape[0]+2, x.shape[1]+2))
    r[1:-1,1:-1] = x
    
    r[0,1:-1] = x[-1]
    r[-1,1:-1] = x[0]
    
    if coords:
        r[:,0,:2] = r[:,1,:2]
        r[:,0,2] = x[0,0,2] - x[0,1,2]
        
        r[:,-1,:2] = r[:,-2,:2]
        r[:,-1,2] = 2*x[0,-1,2] - x[0,-2,2]
    return r

def compute_surface_normals(ds):
    r = {}
    for shape in ds:
        padded_coords = circular_pad(ds[shape]['full_field_coords'][0,...])
        surface_coords = padded_coords
        theta_direction_vector = (surface_coords[2:] - surface_coords[:-2])[:,1:-1]
        z_direction_vector = (surface_coords[:,2:] - surface_coords[:,:-2])[1:-1]
        
        normal_vectors = np.cross(theta_direction_vector, z_direction_vector)
        surface_normals = normal_vectors/np.linalg.norm(normal_vectors,axis=-1,keepdims=True)
        r[shape] = tf.convert_to_tensor(surface_normals)
    return r

def compute_surface_normals2(ds, normalize=True):
    r = {}
    for shape in ds:
        deltas = ds[shape]['full_field_coords'][1] - ds[shape]['full_field_coords'][0]
        if normalize:
            normals = deltas / np.linalg.norm(deltas,axis=-1,keepdims=True)
        else:
            normals = deltas
        r[shape] = tf.convert_to_tensor(normals)
    return r

@tf.function
def surface_integrate(vertex_values, vertex_coords):
    
    padded_coords = tf.concat([vertex_coords, vertex_coords[:1]],0)
    padded_values = tf.concat([vertex_values, vertex_values[:1]],0)
    
    cell_mean_values = tf.nn.conv2d(
        tf.convert_to_tensor([[padded_values]]),
        tf.constant([[[[0.25]],[[0.25]]],[[[0.25]],[[0.25]]]]),
        1, "VALID", data_format="NCHW"
    )[0,0]
    
    z_direction_vector = padded_coords[:-1, 1:] - padded_coords[:-1, :-1]
    theta_direction_vector = -(padded_coords[1:,:-1] - padded_coords[:-1,:-1])
    cell_normals = tf.linalg.cross(theta_direction_vector, z_direction_vector)
    cell_areas = tf.linalg.norm(cell_normals, axis=-1)
    
    return tf.einsum('tzn,tz->n', cell_normals, cell_mean_values)#/(0.5*1.0*(1.0**2)*tf.reduce_sum(cell_areas))

@tf.function
def compute_shear_stresses(velocities, surface_normals, nu=1.0):
    #get surface tangent component of velocity    
    R = tf.constant([[0,-1,0],[1,0,0],[0,0,1]],dtype=velocities.dtype)
    surface_normals_norm = tf.linalg.norm(surface_normals, axis=-1, keepdims=True)
    surface_unit_tangents = tf.einsum('ij,tzj->tzi', R, surface_normals/surface_normals_norm)
    velocity_tangent_component = tf.einsum('tzi,tzi->tz', surface_unit_tangents, velocities[1])
    
    #compute shear stresses from du/dn
    tau = nu*velocity_tangent_component/(2*surface_normals_norm[...,0]**2.0)
    return tau

@tf.function
def compute_friction_forces(shear_stresses, vertex_coords):
    Fnormal = surface_integrate(shear_stresses, vertex_coords)
    return tf.convert_to_tensor([Fnormal[1], -Fnormal[0], Fnormal[2]])

pressure_forces = surface_integrate(
    datasetf['cylinder']['full_field'][600,0,...,0],
    datasetf['cylinder']['full_field_coords'][0,...]
)


print(pressure_forces)
surface_normals_map = compute_surface_normals2(datasetf, normalize=False)
tau = compute_shear_stresses(
    datasetf['cylinder']['full_field'][600,...,1:],
    surface_normals_map['cylinder'],
    0.01
)
friction_forces = compute_friction_forces(tau, datasetf['cylinder']['full_field_coords'][0,...])
print(friction_forces)
#surface_normals_map = compute_surface_normals(datasetf)

#print(surface_normals_map2['cylinder'][:,0])


tf.Tensor([ 45.02644 -18.40995   0.     ], shape=(3,), dtype=float32)
tf.Tensor([9.0921955  0.21526259 0.        ], shape=(3,), dtype=float32)


In [ ]:
models = {}

for v in expt_variables:
    config = json.load(open(experiment_configs[v],'r'))
    model = ConvAutoencoderC(dense_input_units=sensor_shapes[v][1],
                             autoencoder_input_shape=full_field_shapes[v][1:],
                             **config['model'])
    loss_fn = "mse"#tf.keras.losses.get(config['training']['loss'])
    model.compile(l_optimizer= tf.keras.optimizers.get(config['training']['l_optimizer']),
                  loss=loss_fn,
                  optimizer = tf.keras.optimizers.get(config['training']['ae_optimizer']),
                  metrics = config['training'].get('metrics', None))
    model.load_weights(weights_paths[v])
    models[v] = model

array([1, 1, 2, 3, 4, 4])